### Threshold Definition on Detecting the Stress / Rest State

This will focused on getting the How much the metrics drops from the Rest / Stress state for classifying the stress state

In [11]:
## Importing Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [15]:
## Define the path
scipy_rest = os.path.join("Scipy-Rest-HRV_Metrics_UBFC-Phys.csv")
scipy_stress = os.path.join("Scipy-Stress-HRV_Metrics_UBFC-Phys.csv")

elgendi_rest = os.path.join("Elgendi-Rest-HRV-Metrics.csv")
elgendi_stress = os.path.join("Elgendi-Stress-HRV-Metrics.csv")

## Read the CSV files
df_scipy_rest = pd.read_csv(scipy_rest)
df_scipy_stress = pd.read_csv(scipy_stress)

df_scipy_rest.head(),df_scipy_stress.head()

(  Subject    RPPG PR      GT PR  RPPG MeanNN   GT MeanNN    RPPG SDNN  \
 0     s41  81.428530  98.034000   833.962264  617.429124   349.010749   
 1     s42  48.284962  74.299506  1762.000000  838.931075  1158.707225   
 2     s43  68.577569  95.964203  1136.221198  629.002193   718.465097   
 3     s44  67.257494  68.605187  1023.481117  920.673077   372.471869   
 4     s45  55.846749  82.655874  1451.948052  758.109177   927.831207   
 
       GT SDNN   RPPG RMSSD    GT RMSSD  RPPG pNN50   GT pNN50   RPPG LF  \
 0   57.246034   489.743928   70.602253   82.075472  17.525773  0.035133   
 1  157.912389  1732.074611  223.764622   91.000000  61.214953  0.051607   
 2   47.821843   951.342043   67.346224   89.677419   7.368421  0.052770   
 3  183.537949   506.892449  194.124933   91.954023  68.205128  0.054312   
 4  151.220580  1236.128474  229.898547   90.909091  64.556962  0.037576   
 
       GT LF   RPPG HF     GT HF  RPPG LF/HF  GT LF/HF  
 0  0.026856  0.092660  0.019194    0.3

In [16]:
hrv_metrics = [col for col in df_scipy_rest.columns if col not in ['Subject']]

hrv_merged = pd.merge(df_scipy_rest, df_scipy_stress, on='Subject', suffixes=('_rest', '_stress'))

metrics_summary = []

## Metrics summary
for metric in hrv_metrics:
    rest_col = f"{metric}_rest"
    stress_col = f"{metric}_stress"
    
    rest_vals = hrv_merged[rest_col]
    stress_vals = hrv_merged[stress_col]

    # Compute drop percentage
    drop = ((rest_vals - stress_vals) / rest_vals) * 100

    ## Keep only the subjects where the rest > stress (to match the condition)
    valid_mask = rest_vals > stress_vals
    filtered_drop = drop[valid_mask]

    metrics_summary.append({
        "Metric": metric,
        "Subjects_Matched": valid_mask.sum(),
        "Mean_Drop_%": filtered_drop.mean(),
        "Median_Drop_%": filtered_drop.median(),
        "Min_Drop_%": filtered_drop.min(),
        "Max_Drop_%": filtered_drop.max()
    })

# Create a DataFrame with the results
metrics_summary_df = pd.DataFrame(metrics_summary).sort_values(by="Mean_Drop_%", ascending=True)
metrics_summary_df.reset_index(drop=True, inplace=True)

metrics_summary_df.head(20)



,Metric,Subjects_Matched,Mean_Drop_%,Median_Drop_%,Min_Drop_%,Max_Drop_%
0,RPPG pNN50,5,2.498818,1.728228,0.321268,6.003879
1,GT PR,5,3.589826,3.892642,0.743596,6.816332
2,RPPG PR,6,7.787489,9.166752,1.149036,12.718176
3,GT pNN50,3,8.307888,7.701149,5.655656,11.566858
4,GT SDNN,5,10.197536,5.760243,5.442933,22.433697
5,GT MeanNN,10,11.968373,14.050125,1.380659,19.477216
6,GT RMSSD,5,12.798662,8.269543,5.795956,32.604758
7,RPPG MeanNN,11,18.183777,8.730762,0.945703,55.318524
8,GT HF,5,30.198130,22.622750,5.117867,79.303841
9,RPPG RMSSD,10,30.313627,21.936014,0.734553,71.991382
